REcupera las empresas que integran el S&P500

In [1]:
import csv
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
from datetime import datetime

In [2]:
strUrl = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
req = requests.get(strUrl)

In [3]:
#aquí me traigo la página y guardo la primera tabla en un dataframe
soup = BeautifulSoup(req.text, 'lxml')
componentsTable = soup.find_all("table", {"id": "constituents"})
data_components = pd.read_html(str(componentsTable))[0]

In [13]:
data_components.dtypes

Symbol                   object
Security                 object
GICS Sector              object
GICS Sub-Industry        object
Headquarters Location    object
Date added               object
CIK                       int64
Founded                  object
dtype: object

In [15]:
data_components.shape

(503, 8)

In [14]:
data_components.head()

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [16]:
data_components.to_csv('./data/components_data.csv')

Vamos a importar las acciones de algunos de los componentes del índice

In [4]:
#creamos una lista con las empresas a evaluar, vamos a tomar una de cada sector... 11 en total
lstComponentes = ['DIS','BKNG','STZ','COP','SPGI','JNJ','IEX','V','FMC','DLR','XEL','^GSPC']
lstArchivos = ['Disney_data.csv','Bkng_data.csv','Stz_data.csv','Cop_data.csv','Spgi_data.csv','Jnj_data.csv','Iex_data.csv','V_data.csv','Fmc_data.csv','Dlr_data.csv','Xel_data.csv','Gspc_data.csv']

In [5]:
fechaInicio = '2000-02-25'
hoy = datetime.today().strftime('%Y-%m-%d')

for position,instrumento in enumerate(lstComponentes):
    #descarga el intervalo fechaInicio - fechaFin usando yfinance
    ticker = yf.download(instrumento, start=fechaInicio, end=hoy)    

    #guarda los datos del dataframe actualizado
    ticker.to_csv('./data/'+lstArchivos[position])

print('Proceso terminado')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Proceso terminado


Vamos a crear un dataframe con la fecha, y en columnas, los valores de close Adj de cada acción. Este archivo será ocupado mas adelante

In [6]:
#creamos el dataframe vacío
instrumentos_data = pd.DataFrame()

In [7]:
#concatena DAte y close Adj del primer dataframe
data = pd.read_csv('./data/'+lstArchivos[0])
#instrumentos_data['Date',lstComponentes[0]] =
instrumentos_data = data[['Date','Adj Close']]
instrumentos_data.rename({'Adj Close': lstComponentes[0]},axis=1,inplace=True)
instrumentos_data

C:\Users\hmorales\AppData\Local\Temp\ipykernel_1172\4082989128.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  instrumentos_data.rename({'Adj Close': lstComponentes[0]},axis=1,inplace=True)


,Date,DIS
0,2000-02-25,23.985699
1,2000-02-28,25.339733
2,2000-02-29,26.306902
3,2000-03-01,26.838844
4,2000-03-02,26.693764
...,...,...
5783,2023-02-21,102.089996
5784,2023-02-22,101.680000
5785,2023-02-23,101.730003
5786,2023-02-24,100.300003


In [8]:
for position,componente in enumerate(lstComponentes):
    if position >= 1:
        data = pd.read_csv('./data/'+lstArchivos[position])
        instrumentos_data[componente] = data[['Adj Close']] 
        instrumentos_data.rename({'Adj Close': lstComponentes[position]},axis=1,inplace=True)
        
instrumentos_data.head()

,Date,DIS,BKNG,STZ,COP,SPGI,JNJ,IEX,V,FMC,DLR,XEL,^GSPC
0,2000-02-25,23.985699,341.250,5.488533,6.818963,15.235835,19.954931,7.252949,12.727639,4.340192,5.727206,6.742470,1333.359985
1,2000-02-28,25.339733,323.250,5.537851,6.865192,15.256334,20.162794,7.469162,14.495991,4.369244,5.727206,6.910436,1348.050049
2,2000-02-29,26.306902,335.625,5.523761,7.050111,16.691746,19.954931,7.783652,13.455254,4.491256,5.727206,6.742470,1366.420044
3,2000-03-01,26.838844,386.250,5.601263,7.154132,16.445673,20.336016,8.098139,14.248195,4.520309,5.798794,6.598505,1379.189941
4,2000-03-02,26.693764,429.000,5.439213,7.038556,16.507195,20.422628,7.901585,14.408137,4.479635,5.755844,6.550517,1381.760010


In [9]:
instrumentos_data.tail()

,Date,DIS,BKNG,STZ,COP,SPGI,JNJ,IEX,V,FMC,DLR,XEL,^GSPC
5783,2023-02-21,102.089996,2425.489990,223.110001,105.309998,349.878082,158.000000,224.880005,NaN,126.139999,NaN,67.199997,3997.340088
5784,2023-02-22,101.680000,2426.709961,222.119995,104.080002,345.300018,157.779999,224.850006,NaN,128.729996,NaN,66.690002,3991.050049
5785,2023-02-23,101.730003,2426.489990,225.600006,105.430000,348.130005,157.729996,225.690002,NaN,129.080002,NaN,66.330002,4012.320068
5786,2023-02-24,100.300003,2452.479980,224.089996,105.250000,343.380005,155.970001,224.839996,NaN,128.869995,NaN,65.809998,3970.040039
5787,2023-02-27,100.449997,2521.080078,224.690002,105.860001,341.510010,155.630005,225.220001,NaN,129.220001,NaN,65.400002,3982.239990


OJO!!! V y DLR traen valores nulos así que retiramos las columnas

In [10]:
instrumentos_data.drop(['V','DLR'],axis=1,inplace=True)

In [11]:
instrumentos_data.tail()

,Date,DIS,BKNG,STZ,COP,SPGI,JNJ,IEX,FMC,XEL,^GSPC
5783,2023-02-21,102.089996,2425.489990,223.110001,105.309998,349.878082,158.000000,224.880005,126.139999,67.199997,3997.340088
5784,2023-02-22,101.680000,2426.709961,222.119995,104.080002,345.300018,157.779999,224.850006,128.729996,66.690002,3991.050049
5785,2023-02-23,101.730003,2426.489990,225.600006,105.430000,348.130005,157.729996,225.690002,129.080002,66.330002,4012.320068
5786,2023-02-24,100.300003,2452.479980,224.089996,105.250000,343.380005,155.970001,224.839996,128.869995,65.809998,3970.040039
5787,2023-02-27,100.449997,2521.080078,224.690002,105.860001,341.510010,155.630005,225.220001,129.220001,65.400002,3982.239990


In [12]:
#guardamos el dataframe para usarlo en eñlmódulo de analytics
instrumentos_data.to_csv('./data/instrumentos_data.csv')